In [2]:
!pip install keras_facenet
import os
import mmcv
import torch
import cv2 as cv
import numpy as np 
from torch import optim
from PIL import Image, ImageDraw
from IPython import display
from IPython.display import Video
from keras_facenet import FaceNet
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, SubsetRandomSampler
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training

  Using cached keras-facenet-0.3.2.tar.gz (10 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached mtcnn-0.1.1-py3-none-any.whl (2.3 MB)
  Obtaining dependency information for opencv-python>=4.1.0 from https://files.pythonhosted.org/packages/fb/c4/f574ba6f04e6d7bf8c38d23e7a52389566dd7631fee0bcdd79ea07ef2dbf/opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Obtaining dependency information for numpy>=1.21.2 from https://files.pythonhosted.org/packages/72/b2/02770e60c4e2f7e158d923ab0dea4e9f146a2dbf267fec6d8dc61d475689/numpy-1.25.2-cp311-cp311-win_amd64.whl.metadata
Using cached opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl (38.1 MB)
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.5 MB ? eta -:-

ModuleNotFoundError: No module named 'keras_facenet'

In [36]:
class FACELOADING:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (160,160)
        self.X = []
        self.Y = []
        self.detector = MTCNN()
    

    def extract_face(self, filename):
       
        img = cv.imread(filename)
       
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        print("helo") 
        x,y,w,h = self.detector.detect_faces(img)[0]['box']
       
        x,y = abs(x), abs(y)
       
        face = img[y:y+h, x:x+w]
        face_arr = cv.resize(face, self.target_size)
       
        return face_arr
    

    def load_faces(self, dir):
        FACES = []
        for im_name in os.listdir(dir):
            print(im_name)
            try:
                
                path = dir + im_name
               
                single_face = self.extract_face(path)
               
                FACES.append(single_face)
            except Exception as e:
                pass
                print("notdone")
            print(path)
        return FACES

    def load_classes(self):
        for sub_dir in os.listdir(self.directory):
            path = self.directory +'/'+ sub_dir+'/'
            FACES = self.load_faces(path)
            labels = [sub_dir for _ in range(len(FACES))]
            print(path)
            print(f"Loaded successfully: {len(labels)}")
            self.X.extend(FACES)
            self.Y.extend(labels)
        
        return np.asarray(self.X), np.asarray(self.Y)


    def plot_images(self):
        plt.figure(figsize=(18,16))
        for num,image in enumerate(self.X):
            ncols = 3
            nrows = len(self.Y)//ncols + 1
            plt.subplot(nrows,ncols,num+1)
            plt.imshow(image)
            plt.axis('off')
#             Loaded successfully: 0

In [2]:
        !pip install opencv-python  

        img = cv.imread('dataset/apj/OIP(1).jpeg')
        cv.imshow('aa',img)
        waitKey(0)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        x,y,w,h = self.detector.detect_faces(img)[0]['box']
        x,y = abs(x), abs(y)
        face = img[y:y+h, x:x+w]
        face_arr = cv.resize(face, self.target_size)

NameError: name 'cv' is not defined

In [37]:
faceloading = FACELOADING("dataset")
X, Y = faceloading.load_classes()

OIP (1).jpeg
helo
notdone
dataset/apj/OIP (1).jpeg
OIP (2).jpeg
helo
notdone
dataset/apj/OIP (2).jpeg
OIP (3).jpeg
helo
notdone
dataset/apj/OIP (3).jpeg
OIP (4).jpeg
helo
notdone
dataset/apj/OIP (4).jpeg
OIP (5).jpeg
helo
notdone
dataset/apj/OIP (5).jpeg
OIP (6).jpeg
helo
notdone
dataset/apj/OIP (6).jpeg
OIP (7).jpeg
helo
notdone
dataset/apj/OIP (7).jpeg
OIP (8).jpeg
helo
notdone
dataset/apj/OIP (8).jpeg
OIP.jpeg
helo
notdone
dataset/apj/OIP.jpeg
WhatsApp Image 2023-09-05 at 02.20.43.jpg
helo
notdone
dataset/apj/WhatsApp Image 2023-09-05 at 02.20.43.jpg
dataset/apj/
Loaded successfully: 0


In [59]:
plt.figure(figsize=(16,12))
for num,image in enumerate(X):
    ncols = 6
    nrows = len(Y)//ncols + 1
    plt.subplot(nrows,ncols,num+1)
    plt.imshow(image)
    plt.axis('off')

<Figure size 1600x1200 with 0 Axes>

In [60]:

from keras_facenet import FaceNet
embedder = FaceNet()

def get_embedding(face_img):
    face_img = face_img.astype('float32') # 3D(160x160x3)
    face_img = np.expand_dims(face_img, axis=0) 
    # 4D (Nonex160x160x3)
    yhat= embedder.embeddings(face_img)
    return yhat[0] # 512D image (1x1x512)
EMBEDDED_X = []

for img in X:
    EMBEDDED_X.append(get_embedding(img))

EMBEDDED_X = np.asarray(EMBEDDED_X)

In [61]:
from sklearn.preprocessing import LabelEncoder
np.savez_compressed('faces_embeddings_v1.npz', EMBEDDED_X, Y)
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)
plt.plot(EMBEDDED_X[0]) 
plt.ylabel(Y[0])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(EMBEDDED_X, Y, shuffle=True, random_state=17)
model = SVC(kernel='linear', probability=True)
model.fit(X_train, Y_train)
ypreds_train = model.predict(X_train)
ypreds_test = model.predict(X_test)
from sklearn.metrics import accuracy_score

accuracy_score(Y_train, ypreds_train),accuracy_score(Y_test,ypreds_test)

ValueError: Found input variables with inconsistent numbers of samples: [1, 0]

In [ ]:
# t_im = cv.imread(r"C:\Users\sahil\OneDrive\Pictures\Camera Roll\WIN_20230805_02_02_06_Pro.jpg")
# t_im = cv.cvtColor(t_im, cv.COLOR_BGR2RGB)
# x,y,w,h = detector.detect_faces(t_im)[0]['box']
# t_im = t_im[y:y+h, x:x+w]
# t_im = cv.resize(t_im, (160,160))
# test_im = get_embedding(t_im)
# test_im = [test_im]
# ypreds = model.predict(test_im)
# ypreds
# encoder.inverse_transform(ypreds)
##for static image  

In [ ]:
import pickle
with open('svm_model_160x160.pkl','wb') as f:
    pickle.dump(model,f)
# import torch
# model1 = torch.hub.load('ultralytics/yolov5', 'yolov5s')
# cv.imshow('YOLO', np.squeeze(results.render()))      

In [2]:
!pip install keras_facenet

import cv2 as cv
import numpy as np
import os

os.environ['TF_CPP_Mq  IN_LOG_LEVEL']='2'
from sklearn.preprocessing import LabelEncoder
import pickle
from keras_facenet import FaceNet

#INITIALIZE
facenet = FaceNet()
faces_embeddings = np.load("faces_embeddings_v1.npz")
Y = faces_embeddings['arr_1']
encoder = LabelEncoder()
encoder.fit(Y)
haarcascade = cv.CascadeClassifier("face.xml")
model = pickle.load(open("svm_model_160x160.pkl", 'rb'))
cap = cv.VideoCapture(0)
while cap.isOpened():
    _, frame = cap.read()
    frame = cv.flip(frame,flipCode=1)
    rgb_img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    gray_img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = haarcascade.detectMultiScale(gray_img, 1.3, 5)
    for x,y,w,h in faces:
        img = rgb_img[y:y+h, x:x+w]
        img = cv.resize(img, (160,160)) # 1x160x160x3
        img = np.expand_dims(img,axis=0)
        ypred = facenet.embeddings(img)
        face_name = model.predict(ypred)
        final_name = encoder.inverse_transform(face_name)[0]
        cv.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 1)
        cv.putText(frame, str(final_name), (x,y-10), cv.FONT_HERSHEY_SIMPLEX,
                   1, (0,0,255), 3, cv.LINE_AA)
        print(str(final_name))
    cv.imshow("Face Recognition:", frame)
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

DEPRECATION: Loading egg at c:\users\hp\appdata\local\programs\python\python311\lib\site-packages\apache_beam-2.52.0rc4-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\hp\appdata\local\programs\python\python311\lib\site-packages\avro_python3-1.10.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\hp\appdata\local\programs\python\python311\lib\site-packages\contextlib2-21.6.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg a

1/1 [==============================] - 4s 4s/step


ValueError: y contains previously unseen labels: [3]

In [1]:
!pip install keras_facenet

DEPRECATION: Loading egg at c:\users\hp\appdata\local\programs\python\python311\lib\site-packages\apache_beam-2.52.0rc4-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\hp\appdata\local\programs\python\python311\lib\site-packages\avro_python3-1.10.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\hp\appdata\local\programs\python\python311\lib\site-packages\contextlib2-21.6.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg a

In [3]:
import cv2 as cv
import numpy as np
import os
import time

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from sklearn.preprocessing import LabelEncoder
import pickle
from keras_facenet import FaceNet

# Initialize
facenet = FaceNet()
faces_embeddings = np.load("faces_embeddings_v1.npz")
Y = faces_embeddings['arr_1']
encoder = LabelEncoder()
encoder.fit(Y)
haarcascade = cv.CascadeClassifier("face.xml")
model = pickle.load(open("svm_model_160x160.pkl", 'rb'))

# Constants for consistency tracking
CONSISTENCY_INTERVAL = 5  # in seconds
MIN_CONSISTENT_COUNT = 3  # Minimum consistent frames to change color

# Dictionary to store recognized names and their last updated time
recognized_names = {}
name_color = {}

cap = cv.VideoCapture(0)

def update_consistency(name):
    current_time = time.time()
    if name in recognized_names:
        last_time = recognized_names[name]['last_time']
        if current_time - last_time < CONSISTENCY_INTERVAL:
            recognized_names[name]['consistent_count'] += 1
            if recognized_names[name]['consistent_count'] >= MIN_CONSISTENT_COUNT:
                name_color[name] = (0, 255, 0)  # Green
        else:
            recognized_names[name]['consistent_count'] = 0
    else:
        recognized_names[name] = {'last_time': current_time, 'consistent_count': 4}

def draw_bounding_box(frame, x, y, w, h, color, text):
    cv.rectangle(frame, (x, y), (x + w, y + h), color, 1)
    cv.putText(frame, text, (x, y - 10), cv.FONT_HERSHEY_SIMPLEX,
               1, color, 3, cv.LINE_AA)

while cap.isOpened():
    _, frame = cap.read()
    frame = cv.flip(frame, flipCode=1)
    rgb_img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    gray_img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = haarcascade.detectMultiScale(gray_img, 1.3, 5)

    for x, y, w, h in faces:
        img = rgb_img[y:y + h, x:x + w]
        img = cv.resize(img, (160, 160))  # 1x160x160x3
        img = np.expand_dims(img, axis=0)
        ypred = facenet.embeddings(img)
        face_name = model.predict(ypred)
        final_name = encoder.inverse_transform(face_name)[0]

        update_consistency(final_name)

        color = (0, 0, 255)  # Default color is red
        if final_name in name_color:
            color = name_color[final_name]

        draw_bounding_box(frame, x, y, w, h, color, str(final_name))

    cv.imshow("Face Recognition:", frame)

    if cv.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
import cv2 as cv
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import pickle
from keras_facenet import FaceNet
from sklearn.svm import SVC
from torch import optim
from mtcnn.mtcnn import MTCNN
from torch.utils.tensorboard import SummaryWriter
